In [1]:
### Creates new field with the total number of points
# Sums the column "points" in the following time period
#
# Input: CSV file with *one* game
#
# Output: Creates data frame with new columns appended:
#   remaining_time_in_game
#   points_in_window

In [2]:
import pandas as pd

In [8]:
# parameters
INFILE = "C:/Users/kcoltin/Downloads/NBA/work_example_1game.csv"
OUTFILE = "C:/Users/kcoltin/Downloads/NBA/output.csv"
TIME_WINDOW = '180s' # 180s = 3 minutes


In [5]:
# Load file which contains exactly one entire game
dat = pd.read_csv(INFILE)

# QC
print(dat.shape)
print(dat.head())

(431, 49)
   Unnamed: 0   game_id                  data_set       date           a1  \
0           0  20400001  2004-2005 Regular Season  11/2/2004  Jim Jackson   
1           1  20400001  2004-2005 Regular Season  11/2/2004  Jim Jackson   
2           2  20400001  2004-2005 Regular Season  11/2/2004  Jim Jackson   
3           3  20400001  2004-2005 Regular Season  11/2/2004  Jim Jackson   
4           4  20400001  2004-2005 Regular Season  11/2/2004  Jim Jackson   

               a2        a3             a4            a5               h1  \
0  Maurice Taylor  Yao Ming  Tracy McGrady  Charlie Ward  Tayshaun Prince   
1  Maurice Taylor  Yao Ming  Tracy McGrady  Charlie Ward  Tayshaun Prince   
2  Maurice Taylor  Yao Ming  Tracy McGrady  Charlie Ward  Tayshaun Prince   
3  Maurice Taylor  Yao Ming  Tracy McGrady  Charlie Ward  Tayshaun Prince   
4  Maurice Taylor  Yao Ming  Tracy McGrady  Charlie Ward  Tayshaun Prince   

    ...    shot_distance original_x original_y converted_x  conv

In [6]:
# Create column with total remaining time in game
dat['remaining_time_in_game'] = pd.to_datetime(dat['remaining_time']) + pd.to_timedelta((4 - dat['period']) * 12., unit='m')
print(dat[['elapsed', 'remaining_time', 'period', 'remaining_time_in_game']].head())
print(dat[['elapsed', 'remaining_time', 'period', 'remaining_time_in_game']].tail())


   elapsed remaining_time  period remaining_time_in_game
0  0:00:00        0:12:00       1    2020-01-01 00:48:00
1  0:00:00        0:12:00       1    2020-01-01 00:48:00
2  0:00:15        0:11:45       1    2020-01-01 00:47:45
3  0:00:33        0:11:27       1    2020-01-01 00:47:27
4  0:00:42        0:11:18       1    2020-01-01 00:47:18
     elapsed remaining_time  period remaining_time_in_game
426  0:11:36        0:00:24       4    2020-01-01 00:00:24
427  0:11:36        0:00:24       4    2020-01-01 00:00:24
428  0:11:41        0:00:19       4    2020-01-01 00:00:19
429  0:11:43        0:00:17       4    2020-01-01 00:00:17
430  0:12:00        0:00:00       4    2020-01-01 00:00:00


In [18]:
# Create copy of dataframe with rows in reverse order
Rev = dat[::-1] # reverse row order

# Calculate rolling sum - points in *previous* 3 minutes of *time remaining* equivalent to *next* 3 minutes of
# *elapsed time*
Rev.loc[:,'points_lookahead_run'] = Rev[['points', 'remaining_time_in_game']].rolling(window=TIME_WINDOW,
                                                                              on='remaining_time_in_game').sum()['points']

Out = Rev[::-1] # back to original row order

In [21]:
# QC
print(Out[['remaining_time', 'period', 'remaining_time_in_game', 'points', 'points_lookahead_run']].head())
print(Out[['remaining_time', 'period', 'remaining_time_in_game', 'points', 'points_lookahead_run']].tail())

  remaining_time  period remaining_time_in_game  points  points_lookahead_run
0        0:12:00       1    2020-01-01 00:48:00     NaN                   9.0
1        0:12:00       1    2020-01-01 00:48:00     NaN                   9.0
2        0:11:45       1    2020-01-01 00:47:45     2.0                   9.0
3        0:11:27       1    2020-01-01 00:47:27     NaN                   7.0
4        0:11:18       1    2020-01-01 00:47:18     NaN                   7.0
    remaining_time  period remaining_time_in_game  points  \
426        0:00:24       4    2020-01-01 00:00:24     NaN   
427        0:00:24       4    2020-01-01 00:00:24     NaN   
428        0:00:19       4    2020-01-01 00:00:19     0.0   
429        0:00:17       4    2020-01-01 00:00:17     NaN   
430        0:00:00       4    2020-01-01 00:00:00     NaN   

     points_lookahead_run  
426                   0.0  
427                   0.0  
428                   0.0  
429                   NaN  
430                   NaN

In [22]:
Out.to_csv(OUTFILE, index=False)